In [20]:
import numpy as np

from background_models import phi_g_egb_fermi, phi_e_bg_dampe
from finite_clump_vectorized import rho_s_dampe, luminosity, phi_g, rho
from finite_clump_vectorized import gamma_ray_extent, line_width_constraint, mass
from finite_clump_vectorized import fermi_point_src_contraint
from finite_clump_vectorized import anisotropy_integrated, line_width_constraint_chi2
from utilities import fermi_psf, fermi_psf_solid_angle
from utilities import fermi_pt_src_sens_0_0
from utilities import rho_earth, e_high_excess

## Example for NFW profile

All functions are vectorized over $(d, r_s, \gamma)$. They can also take a DM mass, which defaults to $1513.6~\mathrm{GeV}$. $\langle \sigma v \rangle$ is fixed to the usual value.

In [2]:
gamma_nfw = 0.5
dist = np.logspace(-3, 0, 2)
r_s = np.logspace(-3, 1, 2)
dist_mg, r_s_mg = np.meshgrid(dist, r_s)

Density normalization. **This is an input for everything below.**

In [3]:
rho_s_nfw = np.vectorize(rho_s_dampe)(dist_mg, r_s_mg, gamma_nfw, "nfw")

/Users/acoogan/.pyenv/versions/2.7.13/lib/python2.7/site-packages/scipy/integrate/quadpack.py:860: IntegrationWarning: The maximum number of subdivisions (50) has been achieved.
  If increasing the limit yields no improvement it is advised to analyze 
  the integrand in order to determine the difficulties.  If the position of a 
  local difficulty can be determined (singularity, discontinuity) one will 
  probably gain from splitting up the interval and calling the integrator 
  on the subranges.  Perhaps a special-purpose integrator should be used.
  **opt)
/Users/acoogan/.pyenv/versions/2.7.13/lib/python2.7/site-packages/scipy/integrate/quadpack.py:860: IntegrationWarning: The algorithm does not converge.  Roundoff error is detected
  in the extrapolation table.  It is assumed that the requested tolerance
  cannot be achieved, and that the returned result (if full_output = 1) is 
  the best which can be obtained.
  **opt)


Halo mass. Integrates out to $r_{\mathrm{vir}}$ for NFW.

In [4]:
masses_nfw = mass(r_s_mg, rho_s_nfw, gamma_nfw, "nfw")

Local density enhancement $\frac{\rho_s + \rho_\oplus}{\rho_\oplus}$

In [10]:
rho_rel_nfw = (rho(dist_mg, r_s_mg, rho_s_nfw, gamma_nfw, "nfw") + rho_earth) / rho_earth

Luminosity

In [12]:
lum_nfw = luminosity(r_s_mg, rho_s_nfw, gamma_nfw, "nfw")

$J$ factor

In [13]:
from finite_clump_vectorized import J_factor
J_nfw = J_factor(dist_mg, r_s_mg, rho_s_nfw, gamma_nfw, "nfw", fermi_psf)

Gamma-ray flux at energy where clump would be detectable for $m_{\mathrm{DM}} = 1513.6~\mathrm{GeV}$, $\frac{d\phi_\gamma}{d E_\gamma} (E_\gamma = 230$ GeV)

In [14]:
e_gamma_ref = 230.  # GeV
phi_g_nfw = phi_g(e_gamma_ref, dist_mg, r_s_mg, rho_s_nfw, gamma_nfw, "nfw",
                  fermi_psf)

Spatial extent of gamma-ray emission normalized to the size of Fermi's PSF, $\frac{\theta_{68\%}}{\theta_{\mathrm{Fermi}}}$. This is quite slow.

In [15]:
extent_nfw = gamma_ray_extent(dist_mg, r_s_mg, rho_s_nfw, gamma_nfw, "nfw",
                              e_gamma_ref, thresh=0.68)

finite_clump_vectorized.py:186: IntegrationWarning: The maximum number of subdivisions (50) has been achieved.
  If increasing the limit yields no improvement it is advised to analyze 
  the integrand in order to determine the difficulties.  If the position of a 
  local difficulty can be determined (singularity, discontinuity) one will 
  probably gain from splitting up the interval and calling the integrator 
  on the subranges.  Perhaps a special-purpose integrator should be used.
  J_near, err_near = quad(dJ_dr, 0., dist, args, points=points_near, epsabs=0, epsrel=1e-5)
finite_clump_vectorized.py:187: IntegrationWarning: The maximum number of subdivisions (50) has been achieved.
  If increasing the limit yields no improvement it is advised to analyze 
  the integrand in order to determine the difficulties.  If the position of a 
  local difficulty can be determined (singularity, discontinuity) one will 
  probably gain from splitting up the interval and calling the integrator 
  on

Line width constraint. The first function is what we used in the paper: it computes the significance of the largest $e^\pm$ excess over bins excluding the one with the excess. If this is greater than $3$, we say the $e^\pm$ line is too wide.

The second computes the $\chi^2$ test statistic over those bins. Both functions take the argument `excluded_idxs`, which specifies other bins to exclude in the calculation.

Both of these are slow.

In [16]:
lw_nfw = line_width_constraint(dist_mg, r_s_mg, rho_s_nfw, gamma_nfw, "nfw")
lw_chi2_nfw = line_width_constraint_chi2(dist_mg, r_s_mg, rho_s_nfw, gamma_nfw, "nfw")

Fermi point source non-detection constraint. The first function call uses the default sensitivity function, which assumes the clump is located at $(b, \ell) = (120^\circ, 45^\circ)$. The second assumes $(b, \ell) = (0, 0)$.

In [17]:
rho_s_fermi_ps_nfw = fermi_point_src_contraint(dist_mg, r_s_mg, gamma_nfw, "nfw")
rho_s_fermi_ps_0_0_nfw = fermi_point_src_contraint(
    dist_mg, r_s_mg, gamma_nfw, "nfw", fermi_pt_src_sens=fermi_pt_src_sens_0_0)

Extended source constraints. The first line sets the angular radius of the observing cone. The second computes the flux from the cone. The third normalizes to the extragalactic background.

In [21]:
th_ref = 1. * np.pi / 180  # 1 degree
phi_g_ext_nfw = phi_g(
    e_high_excess, dist_mg, r_s_mg, rho_s_nfw, gamma_nfw, "nfw", th_ref)
phi_g_ext_rel_nfw = phi_g_ext_nfw / phi_g_egb_fermi(e_high_excess)

Compute anisotropy (currently seems to not be working). This is very slow due to the complicated integrations. Luckily, the anisotropy is never as important as other constraints, at least for the DM mass/clumps considered in the spike paper.

In [23]:
# e_low, e_high = e_low_aniso_fermi[-1], e_high_aniso_fermi[-1]

# try:
#     anisos_nfw = anisotropy_integrated(
#         e_low, e_high, dist_mg, r_s_mg, rho_s_nfw, gamma_nfw, "nfw", delta_d_rel=1e-4)
# except:
#     print("anisotropy calculation failed")